In [1]:
%%capture

!pip install ffmpeg-python pillow

!git clone https://github.com/soCzech/TransNetV2.git

%cd TransNetV2/inference

!pip install --upgrade ffmpeg-python

In [2]:
!pip install ffmpeg-python

In [3]:
# Import module

import os

import cv2

import json

import glob

import ffmpeg

import torch

import numpy as np

from tqdm import tqdm

from transnetv2 import TransNetV2

from tqdm import notebook

import pandas as pd

In [4]:
model = TransNetV2()

[TransNetV2] Using weights from /kaggle/working/TransNetV2/inference/transnetv2-weights/.


In [25]:
def extract_keyframes(video_path, output_base_dir, map_keyframe_dir):

    _, single_frame_predictions, _ = model.predict_video(video_path)

    scenes = model.predictions_to_scenes(single_frame_predictions)
    video_name = os.path.basename(video_path).split('.')[0]
    output_dir = os.path.join(output_base_dir, video_name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    cap = cv2.VideoCapture(video_path)
    frames = []
    map_dir = f'/kaggle/input/map-keyframe-aic-2024/map-keyframes/{video_name}.csv'
    d = pd.read_csv(map_dir)
    k = d['frame_idx'].to_list()

    for idx, scene in enumerate(scenes):
        start_frame, end_frame = scene
        flag = False
        for x in k:
            if x >= start_frame and x <= end_frame:
                flag = True
                break
        if flag:
            continue

        middle_frame = int((start_frame + end_frame)/2)
        frame_indices = [middle_frame]
        frame_indices.sort()
        for frame_index in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
            ret, frame = cap.read()
            if ret:
                frames.append((frame_index, frame))

    frames_output_dir = output_dir
    if not os.path.exists(frames_output_dir):
        os.makedirs(frames_output_dir)

    df = pd.DataFrame(columns=["n", "frame_idx"])
    i = 0
    for (frame_index, frame) in frames:
        i+= 1
        output_path = os.path.join(frames_output_dir, f"{i:03}.jpg")
        cv2.imwrite(output_path, frame)
        new_row = pd.DataFrame({"n": [i], "frame_idx": [frame_index]})
        df = pd.concat([df, new_row], ignore_index=True)

    csv_file_path = os.path.join(map_keyframe_dir, f"{video_name}.csv")
    df.to_csv(csv_file_path, index=False)
    return video_name, output_dir

In [26]:
all_video_paths = dict()
video_dir = "/kaggle/input/video-v3-aic2024"
for part in sorted(os.listdir(video_dir)):
    parts = part.split('_')
    data_part = parts[-2] + "_" + parts[-1] #if 'L26' in parts[-2] else parts[-2]
    all_video_paths[data_part] =  dict()

for data_part in sorted(all_video_paths.keys()):
    data_part_path = f'{video_dir}/Videos_{data_part}/video'
    video_paths = sorted(os.listdir(data_part_path))
    video_ids = [video_path.replace('.mp4', '').split('_')[-1] for video_path in video_paths]
    for video_id, video_path in zip(video_ids, video_paths):
        video_path_full = f'{data_part_path}/{video_path}'
        all_video_paths[data_part][video_id] = video_path_full

In [27]:
all_video_paths.keys()

dict_keys(['L25_a', 'L26_a', 'L26_b', 'L26_c', 'L26_d', 'L26_e', 'L27_a', 'L28_a', 'L29_a', 'L30_a'])

In [37]:
save_dir = f'/kaggle/working/Keyframes-extra/'
map_keyframe_dir = '/kaggle/working/map-keyframes-extra'
if not os.path.exists(save_dir):
        os.mkdir(save_dir)

if not os.path.exists(map_keyframe_dir):
        os.mkdir(map_keyframe_dir)

for key in tqdm(all_video_paths.keys()):
    video_paths_dict = all_video_paths[key]
    video_ids = sorted(video_paths_dict.keys())
    for video_id in tqdm(video_ids):
        video_path = video_paths_dict[video_id]
        save_path = f'{save_dir}/Keyframes_{key}/keyframes'
        extract_keyframes(video_path, save_path, map_keyframe_dir)

  0%|          | 0/1 [00:00<?, ?it/s]

[TransNetV2] Extracting frames from /kaggle/input/video-v3-aic2024/Videos_L26_a/video/L26_V002.mp4
[TransNetV2] Processing video frames 7447/7447



  0%|          | 0/1 [00:00<?, ?it/s]

[TransNetV2] Extracting frames from /kaggle/input/video-v3-aic2024/Videos_L26_b/video/L26_V100.mp4
[TransNetV2] Processing video frames 8135/8135



100%|██████████| 2/2 [00:36<00:00, 18.45s/it]


In [38]:
!rm -rf /kaggle/working/TransNetV2

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [10]:
print('Completed')

Completed
